In [3]:
import sys
sys.path.append('.')

import faiss
import torch
import numpy as np
from sentence_transformers import SentenceTransformer, util
import nltk
from src.classes.document import Document
from pydantic_core import from_json

ModuleNotFoundError: No module named 'src'

In [ ]:
CORPUS_LOCATION = "corpus_jsonl/corpus.jsonl"

with open(CORPUS_LOCATION) as corpus_file:
    lines = corpus_file.readlines()
    documents = [Document.model_validate_json(line) for line in lines]

k = 100

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

In [ ]:
nltk.download('punkt')
nltk.download('punkt_tab')

In [ ]:
doc_ids = []
corpus_sentences = []
for doc in documents:
    doc_sentences = nltk.sent_tokenize(doc.contents)
    doc_ids.extend([doc.id]*len(doc_sentences))
    corpus_sentences.extend(doc_sentences)
doc_ids_np = np.array(doc_ids)
corpus_sentences_np = np.array(corpus_sentences)
    

In [ ]:
model = SentenceTransformer("all-MiniLM-L6-v2", device=device)
embeddings = np.array(model.encode(corpus_sentences_np,convert_to_tensor=True,batch_size=128,device=device).to('cpu'))

In [ ]:
faiss.normalize_L2(embeddings)
quantizer = faiss.IndexFlatL2(embeddings.shape[1])
index = faiss.IndexIVFFlat(quantizer,embeddings.shape[1],embeddings.shape[0] // k)
index.train(embeddings)
index.add(embeddings)

In [ ]:
query = np.array(model.encode(nltk.sent_tokenize("robot going to 3d imaging. make better motions")))
D, I =index.search(query, 10)
np.unique(I.flatten())


In [ ]:
top_doc_ids = np.unique(doc_ids_np[I.flatten()])
corpus_sentences_np[doc_ids_np == top_doc_ids[0]]
doc_sims = []
for id in top_doc_ids:
    doc_sims.append((str(id),float(max([model.similarity(query_s,embedding)  for query_s in query for embedding in embeddings[doc_ids_np==id]])))) 
doc_sims
sorted(doc_sims, key=lambda x: x[1], reverse=True)

In [ ]:
faiss.write_index(index,"indicies/colbert_index/index.faiss")
np.savez_compressed("indicies/colbert_index/embeddings", embeddings = embeddings, doc_ids = doc_ids_np, sentences = corpus_sentences_np)
""" np.save("indicies/colbert_index/doc_ids", doc_ids_np, allow_pickle=True)
np.save("indicies/colbert_index/sentences", corpus_sentences_np, allow_pickle=True) """

In [ ]:
np_items = np.load("indicies/colbert_index/embeddings.npz")
np_items['sentences']